# Segmenting and Clustering Neighborhoods in Toronto 3

## Clustering Analysis

In [1]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [4]:
df = pd.read_csv('toronto_latlng.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,Postalcode,Borough,Neighborhood,latitude,longitude
0,0,0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,3,3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,4,4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


In [3]:
print('The toronto dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The toronto dataframe has 11 boroughs and 103 neighborhoods.


In [5]:
address = 'Toronto, Ontario Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto Canada are 43.653963, -79.387207.


#### a. Generate maps to visualize the neighborhoods in Toronto

In [7]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['latitude'], df['longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)

In [8]:
map_toronto

#### b. Define Foursquare Credentials and Version

In [9]:
CLIENT_ID = '3IKGELQIJ1BEXQBB1LPG3I15OU53N4ESL4QEOIQVNIBDVIOM' # your Foursquare ID
CLIENT_SECRET = '1SMG1AMAOZGWAMYSOSFM2PXUPINGKVLH3LGSKHABGT1MN3UZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3IKGELQIJ1BEXQBB1LPG3I15OU53N4ESL4QEOIQVNIBDVIOM
CLIENT_SECRET:1SMG1AMAOZGWAMYSOSFM2PXUPINGKVLH3LGSKHABGT1MN3UZ


In [10]:
# Get the neighborhood's name
nei_lat = df.loc[0,'latitude']
nei_lng = df.loc[0,'longitude']
nei_name = df.loc[0,'Neighborhood']
print('Latitude and longitude values of {} are {}, {}.'.format(nei_name, 
                                                               nei_lat, 
                                                               nei_lng))

Latitude and longitude values of Rouge, Malvern are 43.811525000000074, -79.19551721399995.


First, create the get request URL.

In [11]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    nei_lat, 
    nei_lng, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=3IKGELQIJ1BEXQBB1LPG3I15OU53N4ESL4QEOIQVNIBDVIOM&client_secret=1SMG1AMAOZGWAMYSOSFM2PXUPINGKVLH3LGSKHABGT1MN3UZ&v=20180605&ll=43.811525000000074,-79.19551721399995&radius=500&limit=100'

In [13]:
import json
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ca682eb4c1f67161862500c'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.81602500450008,
    'lng': -79.18929289050836},
   'sw': {'lat': 43.80702499550007, 'lng': -79.20174153749154}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5c53f7e18194fc002c76cfbb',
       'name': 'Canadian Appliance Source Whitby',
       'location': {'address': '150 Taunton Rd. W',
        'lat': 43.808353360406464,
        'lng': -79.19133067131041,
        'labeledLatL

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
toronto_df = df
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['latitude'],
                                   longitudes=toronto_df['longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [27]:
print(toronto_venues.shape)
toronto_venues.head()

(2452, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.811525,-79.195517,Canadian Appliance Source Whitby,43.808353,-79.191331,Home Service
1,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
2,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.765690,-79.175256,The Strawberry Patch,43.764738,-79.173081,Tea Room
4,"Guildwood, Morningside, West Hill",43.765690,-79.175256,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping


#### c.Analyze each borough neighborhood

In [28]:
tor = pd.get_dummies(toronto_venues[['Venue Category']], prefix = " ", prefix_sep=" ") 
tor['Neighbourhood'] = toronto_venues['Neighbourhood']

fixed_columns = [tor.columns[-1]] + list(tor.columns[:-1])
tor = tor[fixed_columns]

In [29]:
tor_grouped = tor.groupby('Neighbourhood').mean().reset_index()
tor_grouped

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,...,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.00,0.000000,0.000000,0.0,0.030000,0.000000,0.0,0.010000,0.000000,...,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.0,0.000000,0.000000,0.000000
1,Agincourt,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.0,0.000000,0.000000,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,"Alderwood, Long Branch",0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
5,Bayview Village,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
6,"Bedford Park, Lawrence Manor East",0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
7,Berczy Park,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.015873,0.000000,...,0.000000,0.015873,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
8,"Birch Cliff, Cliffside West",0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
9,"Bloordale Gardens, Eringate, Markland Wood, Ol...",0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [30]:
num_top_venues = 5
for neigh in tor_grouped['Neighbourhood']:
    print("----"+neigh+"----")
    temp = tor_grouped[tor_grouped['Neighbourhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
           venue  freq
0           Café  0.06
1    Coffee Shop  0.06
2          Hotel  0.06
3     Restaurant  0.03
4         Bakery  0.03


----Agincourt----
                  venue  freq
0         Shopping Mall  0.12
1    Chinese Restaurant  0.12
2                Bakery  0.06
3        Discount Store  0.06
4          Skating Rink  0.06


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  freq
0             Pharmacy  0.67
1     Sushi Restaurant  0.33
2    Accessories Store  0.00
3          Music Venue  0.00
4               Museum  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                   venue  freq
0          Grocery Store  0.15
1    Fried Chicken Joint  0.08
2    Japanese Restaurant  0.08
3         Sandwich Place  0.08
4             Beer Store  0.08


----Alderwood, Long Branch----
                     venue  freq
0              C

#### d.Clustering neighborhoods

In [31]:
# set number of clusters
kclusters = 10

tor_grouped_clustering = tor_grouped.drop('Neighbourhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
print(len(kmeans.labels_))

[3 8 6 8 3 2 3 3 3 8]
101


In [ ]:
toronto_merged = df

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!


#### e.Visualize the cluster

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'],kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters